### 검증 세트 (Validation set), 개발세트
- 모델 개발에 데이터를 training과 test set의 2개로 구분하여 사용
- 그러나 이 경우 test set에 오버피팅이 될 가능성이 있음! -> 실전 테스트셋 필요
- 교차 검증 과정`cross validataion`이 필요
- test셋이 역할을 한 training 모델에 대한 최적의 하이퍼파라미터 찾기는 calidation세트가 맡고
- test셋은 현장 데이터 역할을 함 
- 3개의 데이터셋으로 구분하여 사용
- 최적의 하이퍼파라미터를 찾은 후에는 80퍼로 최종 모델 생성 후 테스트한다

### K- Fold 교차검증 Cross Validation
- 데이터의 크기에 따라서 데이터셋(훈련, 검증, 테스트)크기는 조절 필요
- 전체 데이터셋이 작은 경우, 검증 데이터에 별도 20% 할당이 부담스러움!
- 모델 훈련시 일정 부분(fold)로 분할해서 훈련과 검증에 동시 사용(k-fold)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import numpy as np

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()


# 훈련(검증), 테스트셋
train_input, test_input, train_target, test_target = train_test_split(data, target, random_state=42, test_size=0.2)
# 훈련, 검증셋
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, random_state=42, test_size=0.2)

# K-Fold 교차 검증
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
dt.fit(train_input, train_target)

# 사이킷런 model_selection 밑에 교차검증
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
scores

# {'fit_time': array([0.00300026, 0.00299835, 0.00400233, 0.00303411, 0.00300026]),
#  'score_time': array([0.        , 0.        , 0.        , 0.00100112, 0.00100803]),
#  'test_score': array([0.84230769, 0.83365385, 0.84504331, 0.8373436 , 0.8479307 ])}

# 검증 점수
np.mean(scores['test_score'])

0.8412558303102096

### 분할기 사용한 교차검증
- fold 은 기본으로 5이고 
- 분류모델은 `cv =StratifiedKFold()` 회귀모델은 `cv = Kfold()`
- 분류모델은 cv에 설정을 하지 ㅇ낳아도 객체 따라서 자도으로 함수 선택사용 
- cv 인자를 이용하여 원하는 개수로 설정 가능

In [2]:
from sklearn.model_selection import StratifiedKFold

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
np.mean(scores['test_score'])
# ????

0.8335549132947977

### 그리드 검색 (grid search)
- 최적의 하이퍼파라미터를 찾기 위해 반복적으로 변경해가면서 실행
- 여러개 매개변수가 있다면 조합이 급증 됨 -  max_depth, min_impurity_decrease등
- 반복문을 이용해서 찾을 수도 있지만 그리드가 좀 더 편함
- GridSearchCV 클래스는 교차검증을 하면서 하이퍼파라미터를 찾아준다
- 파라미터는 딕셔너리로 설정

In [3]:
# 그리드 검색
from sklearn.model_selection import GridSearchCV

# 매개변수에 노드를 분할하기 위한 불순도 최소량 값을 리스트로 지정
params = {'min_impurity_decrease' : [0.0001, 000.2, 0.0003, 0.0004, 0.0005]}

# 첫번째 인자는 클래스 객체, 기본이 5-fold이고 리스트에 5개 값이 5*5=25개 모델 생성됨, n_jobs는 core개수(-1은 모두 사용)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs = -1)
gs.fit(train_input, train_target)

# 최적의 파라미터 저장
dt = gs.best_estimator_
# 최적의 파라미터로 훈련한 점수
dt.score(train_input, train_target)

# 최적의 파라미터 값
gs.best_params_
#0.001

# 교차 검증
gs.cv_results_['mean_test_score']
# [0.86819297, 0.75793737, 0.86492226, 0.86780891, 0.86761605]

array([0.86819297, 0.75793737, 0.86492226, 0.86780891, 0.86761605])

In [4]:
# 여러개 파라미터 설정
params ={ 'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001), 
          'max_depth' : range(5,20,1),
          'min_samples_split' : range(2,100,10)
        }

gs = GridSearchCV(DecisionTreeClassifier(random_state =42), params, n_jobs=-1)
gs.fit(train_input, train_target)

gs.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

### 랜덤 검색  - 확률 분포 선택
- 랜덤서치를 사용하기 위해서는 scypy에 있는 확률 분포 클래스 사용
- 그리드 서치는 일정 간격 샘플을 사용해 넓은 범위의 경우 많은 모델 생성
- 랜덤서치는 랜덤 샘플을 이용해서 매개 변수로 이용할 것이기 때문에 균등 분포 랜덤값이 필요함
  - 특히 매개변수가 실수일 경우에 더욱 필요
- 균일(균등) 분포 샘플링 클래스 : 실수 uniform 정수 randint

In [5]:
# 랜덤 검색
from scipy.stats import uniform, randint

# 정수 0~9
rgen = randint(0,10)
#rvs(random variable sampling) 10개 생성
rgen.rvs(10)

# 1000개 unique()발생히면 9개의 비슷한 수인 고유값 반환
np.unique(rgen.rvs(1000), return_counts=True)

ugen = uniform(0,1)
ugen.rvs(10)


# 딕셔너리로 매개 변수들의 번위 설정(실수, 정수)
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth' : randint(20,50),
          'min_samples_split' : randint(2,25),
          'min_samples_leaf' : randint(1,25)
          }

# 랜럼 클래스는 교차 검증을 하면서 매개변수 찾음
from sklearn.model_selection import RandomizedSearchCV

# n_iter는 파람스에서 샘플링 100개의 모델을 만듦
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state =42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

gs.best_params_
# {'max_depth': 39,
#  'min_impurity_decrease': 0.00034102546602601173,
#  'min_samples_leaf': 7,
#  'min_samples_split': 13}

# 훈련세트 평가 최대 점수
np.max(gs.cv_results_['mean_test_score'])

# 훈련세트 전체로 데이터 학습
dt = gs.best_estimator_
# 테스트 세트 평가
dt.score(test_input, test_target)

0.86

In [6]:
## 확인 문데
# RandomizedSearchCV에서 디시전트 클래스에 splitter='random' 매개 변수를 추가하고 다시 훈련
# Splitter 매개 변수 기본 값은 best로 각 노드에서 최선의 분할을 찾음
# random으로 하면 무작위로 분할한 다음 가장 좋은 것을 고름
# 테스트 세트 성능이 올라감?

from scipy.stats import uniform, randint

# 파람스 설정
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth' : randint(20,50),
          'min_samples_split' : randint(2,25),
          'min_samples_leaf' : randint(1,25)
          }

# 랜덤서치 함수
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter ='random', random_state =42), params, n_jobs =-1, n_iter =100)
gs.fit(train_input, train_target)

# 베스트 파람스 확인
gs.best_params_

# 베스트인 얘들로 학습
dt = gs.best_estimator_
dt.score(test_input, test_target)
# 떨어짐

0.7884615384615384